---
## 1. Get the data

**Download the data by executing the code below:**

`Notes:`
* This script will download all PDF files from an AWS S3 bucket to a local directory while maintaining the directory structure.
* Ensure you have the necessary AWS credentials and configurations set in a .env file.
* The script uses boto3 to interact with S3 and pathlib to handle file paths.
* If the destination directories do not exist, they will be created automatically.


In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 
sys.path.append(project_root)

from src import data_utils

# Llamar a la función para descargar los PDFs
data_utils.download_pdfs()

Downloaded: nasdaq_annual_reports/.DS_Store to /home/yohana/Documentos/Github/aws_s3_data_downloader/data/nasdaq_annual_reports/.DS_Store
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2019.pdf to /home/yohana/Documentos/Github/aws_s3_data_downloader/data/nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2019.pdf
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2020.pdf to /home/yohana/Documentos/Github/aws_s3_data_downloader/data/nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2020.pdf
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2021.pdf to /home/yohana/Documentos/Github/aws_s3_data_downloader/data/nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2021.pdf
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2022.pdf to /home/yohana/Documentos/Github/aws_s3_data_downloader/data/nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2022.pdf
Downloaded: nasdaq_annual_reports/10x-genomics-inc/NASDAQ_TXG_2019.pdf to /home/yohana